In [105]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

def load_data(file_path='u.data'):
    prefer = []
    for line in open(file_path, 'r'):
        (userid, movieid, rating, ts) = line.split('\t')
        uid = int(userid)
        mid = int(movieid)
        rat = float(rating)
        prefer.append([uid, mid, rat])
    data = np.array(prefer)
    return data

ratings = load_data()
ratings

array([[1.96e+02, 2.42e+02, 3.00e+00],
       [1.86e+02, 3.02e+02, 3.00e+00],
       [2.20e+01, 3.77e+02, 1.00e+00],
       ...,
       [2.76e+02, 1.09e+03, 1.00e+00],
       [1.30e+01, 2.25e+02, 2.00e+00],
       [1.20e+01, 2.03e+02, 3.00e+00]])

In [106]:
len(np.unique(ratings[:,0])), len(np.unique(ratings[:,1]))

(943, 1682)

In [107]:
# Preprocessing for matrix R
import pandas as pd

data = ratings
train_data, test_data = train_test_split(data,test_size = 0.25, random_state = 42)
NUM_USERS = len(np.unique(ratings[:,0]))+1
NUM_ITEMS = len(np.unique(ratings[:,1]))+1
print('dataset density:{:f}'.format(len(data)*1.0/(NUM_USERS*NUM_ITEMS)))

R = np.zeros([NUM_USERS, NUM_ITEMS])

for ele in train_data:
    R[int(ele[0]), int(ele[1])] = float(ele[2])
train_R = R

pd.DataFrame(train_R)

dataset density:0.062942


,0,1,2,3,4,5,6,7,8,9,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,3.0,4.0,0.0,3.0,0.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
import numpy as np
from numpy.random import RandomState
import copy

class PMF():

    # initialize some paprameters
    def __init__(self, R, lambda_alpha=1e-2, lambda_beta=1e-2, latent_size=50, momuntum=0.8,
                 lr=0.001, iters=200, seed=None):
        self.lambda_alpha = lambda_alpha
        self.lambda_beta = lambda_beta
        self.momuntum = momuntum
        self.R = R
        self.random_state = RandomState(seed)
        self.iterations = iters
        self.lr = lr
        self.I = copy.deepcopy(self.R)
        self.I[self.I != 0] = 1

        self.U = 0.1*self.random_state.rand(np.size(R, 0), latent_size)
        self.V = 0.1*self.random_state.rand(np.size(R, 1), latent_size)


    def loss(self):
        # the loss function of the model
        loss = 0.5*np.sum(self.I*(self.R-np.dot(self.U, self.V.T))**2) + 0.5*self.lambda_alpha*np.sum(np.square(self.U)) + 0.5*self.lambda_beta*np.sum(np.square(self.V))
        return loss
    
    def predict(self, data):
        index_data = np.array([[int(ele[0]), int(ele[1])] for ele in data], dtype=int)
        u_features = self.U.take(index_data.take(0, axis=1), axis=0)
        v_features = self.V.take(index_data.take(1, axis=1), axis=0)
        preds_value_array = np.sum(u_features*v_features, 1)
        return preds_value_array


    def train(self, train_data=None, vali_data=None):
        '''
        # training process
        :param train_data: train data with [[i,j],...] and this indacates that K[i,j]=rating
        :param lr: learning rate
        :param iterations: number of iterations
        :return: learned V, T and loss_list during iterations
        '''
        train_loss_list = []
        vali_rmse_list = []
        last_vali_rmse = None

        # monemtum
        momuntum_u = np.zeros(self.U.shape)
        momuntum_v = np.zeros(self.V.shape)

        for it in range(self.iterations):
            # derivate of Vi
            grads_u = np.dot(self.I*(self.R-np.dot(self.U, self.V.T)), -self.V) + self.lambda_alpha*self.U

            # derivate of Tj
            grads_v = np.dot((self.I*(self.R-np.dot(self.U, self.V.T))).T, -self.U) + self.lambda_beta*self.V

            # update the parameters
            momuntum_u = (self.momuntum * momuntum_u) + self.lr * grads_u
            momuntum_v = (self.momuntum * momuntum_v) + self.lr * grads_v
            self.U = self.U - momuntum_u
            self.V = self.V - momuntum_v

            # training evaluation
            train_loss = self.loss()
            train_loss_list.append(train_loss)

            vali_preds = self.predict(vali_data)
            vali_rmse = np.sqrt(np.mean(np.square(vali_data[:,2]-vali_preds)))
            vali_rmse_list.append(vali_rmse)

            print('traning iteration:{: d} ,loss:{: f}, vali_rmse:{: f}'.format(it, train_loss, vali_rmse))

            # if last_vali_rmse and (last_vali_rmse - vali_rmse) < 0:
            #     print('convergence at iterations:{: d}'.format(it))
            #     break
            # else:
            #     last_vali_rmse = vali_rmse

        return self.U, self.V, train_loss_list, vali_rmse_list

In [112]:
lambda_alpha = 0.1
lambda_beta = 0.1
latent_size = 30
lr = 3e-5
iters = 100
model = PMF(R=train_R, lambda_alpha=lambda_alpha, lambda_beta=lambda_beta, latent_size=latent_size, momuntum=0.9, lr=lr, iters=iters, seed=1)
print('parameters are: reg_u={:f}, reg_v={:f}, latent_size={:d}, lr={:f}, iters={:d}'.format(lambda_alpha, lambda_beta, latent_size,lr, iters))
U, V, train_loss_list, vali_rmse_list = model.train(train_data=train_R, vali_data=test_data)

parameters are: reg_u=0.100000, reg_v=0.100000, latent_size=30, lr=0.000030, iters=100
traning iteration: 0 ,loss: 494507.554215, vali_rmse: 3.633167
traning iteration: 1 ,loss: 493375.556161, vali_rmse: 3.629037
traning iteration: 2 ,loss: 491695.555733, vali_rmse: 3.622899
traning iteration: 3 ,loss: 489448.562465, vali_rmse: 3.614671
traning iteration: 4 ,loss: 486597.115413, vali_rmse: 3.604202
traning iteration: 5 ,loss: 483087.577663, vali_rmse: 3.591274
traning iteration: 6 ,loss: 478851.726713, vali_rmse: 3.575605
traning iteration: 7 ,loss: 473808.032012, vali_rmse: 3.556857
traning iteration: 8 ,loss: 467862.976918, vali_rmse: 3.534630
traning iteration: 9 ,loss: 460912.780642, vali_rmse: 3.508467
traning iteration: 10 ,loss: 452845.898131, vali_rmse: 3.477855
traning iteration: 11 ,loss: 443546.711994, vali_rmse: 3.442232
traning iteration: 12 ,loss: 432900.866144, vali_rmse: 3.400998
traning iteration: 13 ,loss: 420802.704200, vali_rmse: 3.353533
traning iteration: 14 ,loss

In [113]:
# Constrained PMF
class PMF_2():
  def __init__(self, train_R = None, constrained = False ):
    self.num_feat = 30
    self.momentum = 0.9
    self.learning_rate = 0.0005
    self.itr = 80
    self.lambda_V = 0.001
    self.lambda_U = 0.001
    self.lambda_W = 0.001
    self.train_R = train_R
    self.constrained = constrained

    self.train_I = copy.deepcopy(self.train_R)
    self.train_I[self.train_I>0] = 1

    rand_num1 = np.random.RandomState(100)
    rand_num2 = np.random.RandomState(101)
    
    self.U = rand_num1.randn(self.num_feat,train_R.shape[0])
    self.V = rand_num1.randn(self.num_feat,train_R.shape[1])
    self.W = rand_num2.randn(self.num_feat,train_R.shape[1])

    self.Y = None
    self.sigma_I = np.dot(self.train_I,np.ones(self.train_R.shape[1]))
    self.sigma_I[self.sigma_I == 0] = np.inf


  def loss_function(self):
    if not self.constrained:
      return 0.5*np.sum((self.train_I*(self.train_R-np.dot(self.U.T,self.V))**2)) + 0.5*(self.lambda_U)*np.sum(np.square(self.U)) + 0.5*(self.lambda_V)*np.sum(np.square(self.V))
    else:
      return 0.5 * np.sum(self.train_I*(self.train_R- np.dot(((self.U+(np.dot(self.W,self.train_I.T)/self.sigma_I)).T),self.V))**2) + (self.lambda_U/2)*np.sum((self.U)**2) + (self.lambda_V/2)*np.sum((self.V**2)) + (self.lambda_W/2)*np.sum((self.W)**2)

  
  def predict(self,test_data):
    pred_value = []
    if (self.constrained):
      sigma_I = np.dot(self.train_I,np.ones(self.train_R.shape[1]))
      sigma_I[self.sigma_I == 0] = np.inf
      self.Y = self.U + (np.dot(self.W, self.train_I.T)/sigma_I)
      for locate in test_data:
        pred_value.append(np.dot(self.Y[:,locate[0]],self.V[:,locate[1]]))

    else:
      for locate in test_data:
        pred_value.append(np.dot(self.U[:,locate[0]],self.V[:,locate[1]]))

    return np.array(pred_value)

  def fit(self, test_data, df_matrix):
    train_mse = []
    test_mse = []

    mmt_U = np.zeros(self.U.shape)
    mmt_V = np.zeros(self.V.shape)

    if not self.constrained:
      for i in range (self.itr):
        dv_u = - (np.dot((self.train_I*(self.train_R - np.dot(self.U.T,self.V))),self.V.T)).T + self.lambda_U*self.U
        dv_v = - (np.dot(self.U, (self.train_I*(self.train_R - np.dot(self.U.T, self.V))))) + self.lambda_V*self.V

        mmt_U = (self.momentum * mmt_U) + self.learning_rate * dv_u
        mmt_V = (self.momentum * mmt_V) + self.learning_rate * dv_v
        self.U = self.U - mmt_U * self.momentum
        self.V = self.V - mmt_V * self.momentum
        train_mse_loss = self.loss_function()
        train_mse.append(train_mse_loss)

        test_predicts = self.predict(test_data)
        test_data_values = df_matrix[test_data[:,0],test_data[:,1]]
        test_mse_value =np.sum(np.square(test_data_values-test_predicts))/len(test_data_values)
        test_mse.append(test_mse_value)

        print('traning iteration:{: d} ,loss:{: f}, test_rmse:{: f}'.format(i, train_mse_loss, test_mse_value))


    else:
      mmt_W = np.zeros(self.W.shape)
      for i in range(self.itr):

        dv_u = - (np.dot(self.train_I*(self.train_R-np.dot(self.U.T + (np.dot(self.W,self.train_I.T)/self.sigma_I).T,self.V)),self.V.T)).T + self.lambda_U*self.U
        dv_v = - np.dot(self.U ,(self.train_I*(self.train_R-np.dot(self.U.T + (np.dot(self.W, self.train_I.T)/self.sigma_I).T, self.V)))) + self.lambda_V*self.V
        #dv_v = - (np.dot(self.train_I*(self.train_R-np.dot(self.U.T + (np.dot((self.W,self.train_I.T)/self.sigma_I).T),self.V),self.U.T+(np.dot(self.W,self.train_I.T)/self.sigma_I).T))).T + self.lambda_V*self.V
        dv_w = - (np.dot((self.train_I*(self.train_R-np.dot(self.U.T + (np.dot(self.W,self.train_I.T)/self.sigma_I).T,self.V))).T, np.dot(self.train_I/self.sigma_I.reshape(-1,1),self.V.T))).T + self.lambda_W*self.W

        mmt_U = (self.momentum * mmt_U) + self.learning_rate * dv_u
        mmt_V = (self.momentum * mmt_V) + self.learning_rate * dv_v
        mmt_W = (self.momentum * mmt_W) + self.learning_rate * dv_w
        self.U = self.U - self.momentum * mmt_U
        self.V = self.V - self.momentum * mmt_V
        self.W = self.V - self.momentum * mmt_W
        train_mse_loss = self.loss_function()
        train_mse.append(train_mse_loss)

        test_predicts = self.predict(test_data)
        test_data_values = df_matrix[test_data[:,0],test_data[:,1]]
        test_mse_value =np.sum(np.square(test_data_values-test_predicts))/len(test_data_values)
        test_mse.append(test_mse_value)

        print('traning iteration:{: d} ,loss:{: f}, test_rmse:{: f}'.format(i, train_mse_loss, test_mse_value))



      pass

    return self.U, self.V, train_mse, test_mse

In [116]:
# Preprocess ratings into R
import math
df = pd.DataFrame(ratings, columns=['user', 'item', 'rating'])
df = df.pivot_table(index='user', columns='item', values ='rating')
df_matrix = df.to_numpy()
locate = []
for i in range (len(df_matrix)):
  for j in range (len(df_matrix[0])):
    if not(math.isnan(df_matrix[i][j])):
      locate.append((i,j))
data = np.array(locate)
data = data.astype(int)

train_data, test_data = train_test_split(data,test_size = 0.25, random_state = 42)
train_df = np.zeros(df_matrix.shape)
test_df = np.zeros(df_matrix.shape)

for tup in train_data:
  train_df[tup[0]][tup[1]] = df_matrix[tup[0]][tup[1]]
for tup in test_data:
  test_df[tup[0]][tup[1]] = df_matrix[tup[0]][tup[1]]

print(train_df)

[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [117]:
test_data

array([[692, 381],
       [746, 110],
       [200, 211],
       ...,
       [756,  23],
       [591, 930],
       [108,  97]])

In [119]:
model1 = PMF_2(train_R = train_df, constrained= True)
U1, V1, tr_loss_list1, test_rmse_list1 = model1.fit(test_data, df_matrix)
preds = model1.predict(test_data=test_data)

traning iteration: 0 ,loss: 1255482.448738, test_rmse: 38.111642
traning iteration: 1 ,loss: 887682.362886, test_rmse: 29.780032
traning iteration: 2 ,loss: 631667.140073, test_rmse: 23.373577
traning iteration: 3 ,loss: 492630.371940, test_rmse: 19.301712
traning iteration: 4 ,loss: 416179.282208, test_rmse: 16.599914
traning iteration: 5 ,loss: 356716.518384, test_rmse: 14.325277
traning iteration: 6 ,loss: 300166.238347, test_rmse: 12.169943
traning iteration: 7 ,loss: 267325.679194, test_rmse: 10.684963
traning iteration: 8 ,loss: 268469.495499, test_rmse: 10.240280
traning iteration: 9 ,loss: 251839.876366, test_rmse: 9.605368
traning iteration: 10 ,loss: 208495.808735, test_rmse: 8.407351
traning iteration: 11 ,loss: 184023.017493, test_rmse: 7.636058
traning iteration: 12 ,loss: 178267.772264, test_rmse: 7.278347
traning iteration: 13 ,loss: 170755.637122, test_rmse: 6.866471
traning iteration: 14 ,loss: 153758.234012, test_rmse: 6.238725
traning iteration: 15 ,loss: 131999.9922